<a href="https://colab.research.google.com/github/Sankytanky100/Agents/blob/main/Genomic_Chatbot_(Incomplete).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install "camel-ai[all]==0.2.11"
!pip install starlette
!pip install nest_asyncio

In [1]:
import os
from getpass import getpass

# Prompt for the Firecrawl API key securely
firecrawl_api_key = getpass('Enter your API key: ')
os.environ["FIRECRAWL_API_KEY"] = firecrawl_api_key

Enter your API key: ··········


In [2]:
import os
from getpass import getpass

samba_api_key = getpass('Enter your API key: ')
os.environ["SAMBA_API_KEY"] = samba_api_key

Enter your API key: ··········


In [6]:
import os

# Initialize the Firecrawl instance
firecrawl = Firecrawl()

# Use the `crawl` method to retrieve content from the specified URL
firecrawl_response = firecrawl.crawl(
    url="https://sambanova.ai/blog/qwen-2.5-32b-coder-available-on-sambanova-cloud"  # Target URL to crawl for content
)

print(firecrawl_response["status"])

# Print the markdown content from the first page in the crawled data
print(firecrawl_response["data"][0]["markdown"])

knowledge = firecrawl_response["data"][0]["markdown"]


NameError: name 'Firecrawl' is not defined

In [ ]:
os.makedirs('local_data', exist_ok=True)

with open("local_data/sambanova_announcement.md", 'w') as file:
  file.write(knowledge)

In [ ]:
from camel.configs import SambaCloudAPIConfig
from camel.models import ModelFactory
from camel.types import ModelPlatformType, ModelType
from camel.agents import ChatAgent
from camel.messages import BaseMessage


qwen_model= ModelFactory.create(
    model_platform=ModelPlatformType. SAMBA,
    model_type="Qwen2.5-Coder-328-Instruct",
    model_config_dict=SambaCloudAPIConfig(max_tokens=500).as_dict(),)


chat_agent = ChatAgent(
    system_message="You're a helpful assistant",
    message_window_size=20,
    model=qwen_model
)

In [ ]:
knowledge_message = BaseMessage.make_user_message(
    role_name="User",
    content="Based on the following knowledge: (knowledge)"
)


In [ ]:
print("Start chatting! Type 'exit' to end the conversation.")
while True:
    user_input = input("User: ")
    if user_input.lower() == "exit":
        print("Ending conversation.")
        break
    assistant_response = chat_agent.step(user_input)
    print("Assistant:", assistant_response.msgs[0].content)


# Configuring the ENCODE Dataset

In [7]:
!pip install boto3 botocore



In [8]:
import boto3
import botocore
import io
import gzip

# Create an S3 client configured for unsigned (public) requests
s3_client = boto3.client(
    's3',
    region_name='us-west-2',
    config=botocore.client.Config(signature_version=botocore.UNSIGNED)
)






In [9]:
# List objects in the 'encode-public' bucket
response = s3_client.list_objects_v2(Bucket='encode-public')

for obj in response.get('Contents', []):
    print(obj['Key'])


2008/11/24/034e3689-9903-4c86-9237-040f8f795b73/ENCFF001SNN.broadPeak.gz
2008/11/24/0868284e-8c3c-488d-89e6-487cd89971c3/ENCFF000AAU.broadPeak.bigbed
2008/11/24/0b903d8b-824c-4e34-9b24-a1d23e31d83f/ENCFF001SNC.broadPeak.gz
2008/11/24/0e0d13f7-4e7c-4cee-95a8-e3dc1c1351d1/ENCFF000ABT.broadPeak.bigbed
2008/11/24/0e5cd34b-4a2a-4bd5-a063-15a46fa48016/ENCFF001SMZ.broadPeak.gz
2008/11/24/0ecf15e8-7cf9-4f33-9e1c-71d5b384c12e/ENCFF001SNQ.broadPeak.gz
2008/11/24/0ffbb597-0ddf-4b0a-96b4-390e9d6dc997/ENCFF000ABS.broadPeak.bigbed
2008/11/24/16c640cb-a136-4a9e-b539-7ca412d7de00/ENCFF000ABM.broadPeak.bigbed
2008/11/24/17795607-7209-4d1e-9c0e-2369079984cc/ENCFF000ACL.broadPeak.bigbed
2008/11/24/1be34318-cf70-442e-856c-2135efb84ec1/ENCFF000ABX.broadPeak.bigbed
2008/11/24/1c3d3633-8e63-49ff-a6d3-ce54f35a0585/ENCFF000ACH.broadPeak.bigbed
2008/11/24/1e32d877-8c90-4ed5-aef4-93ecbba6a3f4/ENCFF001SNG.broadPeak.gz
2008/11/24/1e85b0b0-3016-47cb-b9cb-e593bb3958ed/ENCFF001SNJ.broadPeak.gz
2008/11/24/1eacb8d0-cfc

In [16]:
import boto3
import botocore
import io
import gzip

# Define the S3 bucket name and select an object key from your list.
bucket_name = 'encode-public'
# Example file: Replace with your chosen file from the output list.
object_key = '2010/11/16/0c744ab2-e5de-4852-9a93-021fdc6d82f7/ENCFF001TCG.broadPeak.gz'

# Create an S3 client configured for unsigned (public) requests.
s3_client = boto3.client(
    's3',
    region_name='us-west-2',
    config=botocore.client.Config(signature_version=botocore.UNSIGNED)
)

# List objects in the bucket (optional, for reference)
response_list = s3_client.list_objects_v2(Bucket=bucket_name)
print("Objects in the bucket:")
for obj in response_list.get('Contents', []):
    print(obj['Key'])

# Retrieve the selected object from S3.
response = s3_client.get_object(Bucket=bucket_name, Key=object_key)
raw_data = response['Body'].read()

# Check if the file is gzip-compressed based on its extension.
if object_key.endswith('.gz'):
    # Decompress using gzip.
    with gzip.GzipFile(fileobj=io.BytesIO(raw_data), mode='rb') as gz:
        decompressed_data = gz.read()
    # Decode the decompressed data as UTF-8.
    file_content = decompressed_data.decode('utf-8', errors='replace')
else:
    # Directly decode if not compressed.
    file_content = raw_data.decode('utf-8', errors='replace')

print("\nFile content:")
print(file_content)


Objects in the bucket:
2008/11/24/034e3689-9903-4c86-9237-040f8f795b73/ENCFF001SNN.broadPeak.gz
2008/11/24/0868284e-8c3c-488d-89e6-487cd89971c3/ENCFF000AAU.broadPeak.bigbed
2008/11/24/0b903d8b-824c-4e34-9b24-a1d23e31d83f/ENCFF001SNC.broadPeak.gz
2008/11/24/0e0d13f7-4e7c-4cee-95a8-e3dc1c1351d1/ENCFF000ABT.broadPeak.bigbed
2008/11/24/0e5cd34b-4a2a-4bd5-a063-15a46fa48016/ENCFF001SMZ.broadPeak.gz
2008/11/24/0ecf15e8-7cf9-4f33-9e1c-71d5b384c12e/ENCFF001SNQ.broadPeak.gz
2008/11/24/0ffbb597-0ddf-4b0a-96b4-390e9d6dc997/ENCFF000ABS.broadPeak.bigbed
2008/11/24/16c640cb-a136-4a9e-b539-7ca412d7de00/ENCFF000ABM.broadPeak.bigbed
2008/11/24/17795607-7209-4d1e-9c0e-2369079984cc/ENCFF000ACL.broadPeak.bigbed
2008/11/24/1be34318-cf70-442e-856c-2135efb84ec1/ENCFF000ABX.broadPeak.bigbed
2008/11/24/1c3d3633-8e63-49ff-a6d3-ce54f35a0585/ENCFF000ACH.broadPeak.bigbed
2008/11/24/1e32d877-8c90-4ed5-aef4-93ecbba6a3f4/ENCFF001SNG.broadPeak.gz
2008/11/24/1e85b0b0-3016-47cb-b9cb-e593bb3958ed/ENCFF001SNJ.broadPeak.gz


In [1]:
#@title Google Colab ENCODE Files Ingestion Script

# Step 1. Upload the files.txt file (select it from your computer).
from google.colab import files
uploaded = files.upload()  # Upload your files.txt file when prompted


Saving files (3).txt to files (3).txt


In [6]:
import requests
import pandas as pd

# Step 2. Read the list of URLs from the uploaded files.txt.
filename = "files (3).txt"  # Ensure your uploaded file is named "files.txt"
with open(filename, "r") as f:
    urls = [line.strip() for line in f if line.strip()]

if not urls:
    raise ValueError("No URLs found in files.txt")

# Step 3. Download the metadata file (the first URL in the list).
metadata_url = urls[0].strip('"')  # Remove extra quotes if present
print("Metadata URL:", metadata_url)

metadata_response = requests.get(metadata_url)
if metadata_response.status_code == 200:
    with open("metadata.tsv", "wb") as meta_file:
        meta_file.write(metadata_response.content)
    print("Metadata downloaded as 'metadata.tsv'")
else:
    raise Exception(f"Failed to download metadata file. Status code: {metadata_response.status_code}")

# Step 4. Load the metadata.tsv into a Pandas DataFrame.
metadata_df = pd.read_csv("metadata.tsv", sep="\t")
print("Metadata preview:")
print(metadata_df.head())

# Optional Step 5. Download one example experimental file (the second URL in the list).
if len(urls) > 1:
    example_data_url = urls[1].strip('"')  # Remove extra quotes if present
    print("\nDownloading a sample experimental file from:", example_data_url)
    data_response = requests.get(example_data_url)
    if data_response.status_code == 200:
        # Extract the filename from the URL (everything after '@@download/').
        file_name = example_data_url.split("/@@download/")[-1]
        with open(file_name, "wb") as data_file:
            data_file.write(data_response.content)
        print(f"Sample data file saved as '{file_name}'")
    else:
        print(f"Failed to download sample data file. Status code: {data_response.status_code}")

Metadata URL: https://www.encodeproject.org/metadata/?replicates.library.biosample.donor.organism.scientific_name=Homo+sapiens&biosample_ontology.system_slims=central+nervous+system&status=released&type=Experiment
Metadata downloaded as 'metadata.tsv'


<ipython-input-6-a56c43c9e01e>:25: DtypeWarning: Columns (41) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata_df = pd.read_csv("metadata.tsv", sep="\t")


Metadata preview:
  File accession File format File type File format type  \
0    ENCFF688JFN       fastq     fastq              NaN   
1    ENCFF803FBL      bigWig    bigWig              NaN   
2    ENCFF416UGY         bam       bam              NaN   
3    ENCFF482BDQ      bigWig    bigWig              NaN   
4    ENCFF504BTV      bigWig    bigWig              NaN   

                           Output type File assembly Experiment accession  \
0                                reads           NaN          ENCSR634JQK   
1  minus strand signal of unique reads        GRCh38          ENCSR634JQK   
2                           alignments        GRCh38          ENCSR634JQK   
3   plus strand signal of unique reads        GRCh38          ENCSR634JQK   
4     minus strand signal of all reads        GRCh38          ENCSR634JQK   

           Assay                    Donor(s) Biosample term id  ...  \
0  total RNA-seq  /human-donors/ENCDO242JQO/    UBERON:0009834  ...   
1  total RNA-seq  /hum

In [8]:
import gzip

# Open the FASTQ file in text mode (rt means read text)
with gzip.open('ENCFF688JFN.fastq.gz', 'rt') as f:
    # Print the first 40 lines (10 records)
    for i in range(40):
        print(f.readline().strip())


@VH00472:2:AAAWCGGM5:1:1101:18743:1057 1:N:0:CCTTCCAT
CCACTACCGGTGTCCAGACACCAGGATGTCTGTCAAGCAGAGGCCATTTCCAGCATGGTATCTGTGAGGCAGGCGCTGTGAGGAACACATGACACACATGA
+
C-;C-;-CC;CCC-C-CCCCCCCCC--;-!CC;C;CC-C;-CC-;CC-;CCCCCCC;CCCCCC!CCCCCCCC;CCCCCCCCCCCCCCC;C;CCCCCCCCCC
@VH00472:2:AAAWCGGM5:1:1101:19348:1057 1:N:0:CCTTCCAT
CCCCATCGGGAGGCCGAACTGCGTGAGGACACCCGATCGGCCTAGCGCACTACAGCCCAGAACGCCTGGACTCAAGCGATCCTCCAGCCTCAGCCTCCCGA
+
C--;-CC-CC-C;CCCCCC-C;;CC-CCC!CC-C;C;CC--;C-;CC;CCCCC-;CC;CCCCC!CCC;CC;CCCCCCCCCC;CCCCCC-CCCCCCCC-CCC
@VH00472:2:AAAWCGGM5:1:1101:19386:1057 1:N:0:CCTTCCAT
TGTGTGGGGGTGGGGGGGCTGCGGGCACACTGTCTGGGGTGGGTTTGTGGGTGTGGTGGGGGTCTAGGAGAAAAGACAGGGCGAGAGAAAGATGACCAATT
+
CCCC--CCCC;C;CCC;CC;CCCCC-C-C!C-CCC;--;CCC;CCCC;;CCCCCCCC;CCC-;!CCCCCC-CCCCCCC;CCCCCCCCCCCCCCC;CCCCCC
@VH00472:2:AAAWCGGM5:1:1101:19651:1057 1:N:0:CCTTCCAT
CCGGGACGCCAGCGCTGAGTGCTACACCAGTGTCTTTCTAAGATAGCCTGCTTTCCAGCCTAGGAACACTGGTTTTTAGGAGGTTCCCTGAAAACATTTTA
+
C;CC;CCC--CC-CCC;-;CC-;C;CC;C!C--CCCC-CCCC;CC;C-CCCC-CC-C;;CCC

In [10]:
import pandas as pd

# Load metadata.tsv into a Pandas DataFrame.
metadata_df = pd.read_csv("metadata.tsv", sep="\t")

# Display the first few rows of the metadata.
print(metadata_df.head())

print(metadata_df.columns.tolist())


  File accession File format File type File format type  \
0    ENCFF688JFN       fastq     fastq              NaN   
1    ENCFF803FBL      bigWig    bigWig              NaN   
2    ENCFF416UGY         bam       bam              NaN   
3    ENCFF482BDQ      bigWig    bigWig              NaN   
4    ENCFF504BTV      bigWig    bigWig              NaN   

                           Output type File assembly Experiment accession  \
0                                reads           NaN          ENCSR634JQK   
1  minus strand signal of unique reads        GRCh38          ENCSR634JQK   
2                           alignments        GRCh38          ENCSR634JQK   
3   plus strand signal of unique reads        GRCh38          ENCSR634JQK   
4     minus strand signal of all reads        GRCh38          ENCSR634JQK   

           Assay                    Donor(s) Biosample term id  ...  \
0  total RNA-seq  /human-donors/ENCDO242JQO/    UBERON:0009834  ...   
1  total RNA-seq  /human-donors/ENCDO242

<ipython-input-10-06a8f903fd9b>:4: DtypeWarning: Columns (41) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata_df = pd.read_csv("metadata.tsv", sep="\t")


In [17]:


# Extract and display the first 5 rows of these columns:
print(metadata_df[columns_to_show].head(5))


  Experiment accession          Assay                    Donor(s)  \
0          ENCSR634JQK  total RNA-seq  /human-donors/ENCDO242JQO/   
1          ENCSR634JQK  total RNA-seq  /human-donors/ENCDO242JQO/   
2          ENCSR634JQK  total RNA-seq  /human-donors/ENCDO242JQO/   
3          ENCSR634JQK  total RNA-seq  /human-donors/ENCDO242JQO/   
4          ENCSR634JQK  total RNA-seq  /human-donors/ENCDO242JQO/   

         File analysis title File analysis status          dbxrefs  \
0                        NaN                  NaN  SRA:SRR22521289   
1  ENCODE4 v1.2.3 GRCh38 V29             released              NaN   
2  ENCODE4 v1.2.3 GRCh38 V29             released              NaN   
3  ENCODE4 v1.2.3 GRCh38 V29             released              NaN   
4  ENCODE4 v1.2.3 GRCh38 V29             released              NaN   

   Biosample genetic modifications zygosity  
0                                       NaN  
1                                       NaN  
2                         